<h1>WavesPlasma Python Library</h1>

This notebok serves as a library of functions and variable definitions to work in the context of wave propagation in magnetically confined plasmas.

The numerical scheme utilised here is developed, described and explained in Muriel Colin's PhD thesis (2001). The original code was written in Mathematica by Stéphane Heuraux and the current file is the translation (and expansion with auxiliary functions) by Jorge Paz-Peñuelas Oliván in February 2025.

In [1]:
import numpy as np
import cmath as cm

from scipy.fft import fft,fftfreq
from scipy.interpolate import make_smoothing_spline

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import clear_output

import copy
from timeit import default_timer as timer
import pandas as pd

<h4>Importing this notebook</h4>

This notebook has been designed to be imported using the ```import_ipynb``` library. It is  recommended to define the variables written below just after importing (their values can be changed).

<h4>-------</h4>

<h3>Definition of the plasma study domain</h3>

We are going to work with the outside-facing side of an axial cross section of a tokamak, approximated to a cylindrical domain with rotational symmetry around its axis (the axis is parallel to the tokamak's radial direction). The coordinate in the axis direction will be denoted $x$, and $y$ will denote the component in the perpendicular plane.
In the plasma domain we will asign a certain plasma density profile $n_e(x)$, as well as a magnetic field $B(x)$. Our goal is to test the validity of the time-of-flight theoretical WKB-approximated estimations near the plasma edge.

To describe the plasma, instead of $n_e(x)$ and $B(x)$ we will work with the plasma frequency $\omega_{pe}(x)$ and the cyclotron frequency $\omega_c(x)$ as defined below.

$$
\omega_{pe}^2(x)=\frac{n_e(x)e^2}{\varepsilon_0m_e}\ ,\qquad\omega_c(x)=\frac{eB(x)}{m_e}\ .
$$

Numerically, the cavity of interest will be discretised into $N_x$ cells, and we will only consider presence of plasma at $x>P_\text{plasma}$ (```p0plas```), which will be expressed in cell number terms. We will describe the plasma in terms of a characteristic wavenumber $k_{00}$ in units of cells$^{-1}$. By assigning this characteristic wavenumber to a real-world wave frequency $\nu$ we will be able to provide length units to the grid cells. 

The original values in the mathematica code of these 3 variables were ```Nx = 2100```, ```p0plas = 90```, ```k00 = 0.025```.

<h3>Waves in a magnetised plasma</h3>

In a magnetised plasma, an X-mode propagating wave with frequency $\omega$ does so with the following index:

$$N_X^2(x,\omega)=1-\frac{A(1-A)}{1-A-B}\ ,\qquad\text{with }A=\frac{\omega_{pe}^2(x)}{\omega^2}\text{ and }B=\frac{\omega_c^2(x)}{\omega^2}$$

with cut-off condition $A(1-A)=1-A-B$ , or equivalently,

$$\omega^2=\omega_{pe}^2+\frac{1}{2}\omega_c^2\pm\omega_c\sqrt{\omega_{pe}^2+\frac{1}{4}\omega_c^2}$$

Notice that when $\omega_c=0$ we recover the O-mode propagation.

To describe the probing wave we will make use of is wavenumber $k_{i}$. Note that for the wave modelling we will be using what is known as **magic step**, which is simply setting the space and time grid intervals equal ($\Delta x=\Delta y$), meaning that $c=1$. In this context, as $c=\frac{\omega}{k_i}$, we can take $\omega=k_i$.

In [2]:
# DEFINITION OF PLASMA FREQUENCIES DICTIONARIES

def freq_dicts(Nx,k00,p0plas):

    wpe2_ = {
        'default': 0.01*np.array([k00**2*(x-p0plas)/Nx if x>p0plas else 0 for x in range(Nx)]),
        'vacuum': np.zeros(Nx),
        'constant': np.array([0.1*k00**2 if x>=10 else 0 for x in range(Nx)]),
        'dense': np.array([k00**2*(x-p0plas)/(Nx-2*p0plas) if x>p0plas else 0 for x in range(Nx)]),
        'buffer': np.array([k00**2*(x-p0plas)/(Nx-5*p0plas) if x>p0plas else 0 for x in range(Nx)]),
        'parabolic': np.array([k00**2*(x-p0plas)**2/(Nx-2*p0plas)**2 if x>p0plas else 0 for x in range(Nx)]),
    }

    wc_ = {
        'default': np.array([k00/(1-(x-p0plas)*1.6/200000) for x in range(Nx)]),
        'vacuum': np.zeros(Nx)
    }
    
    return wpe2_,wc_



# PLASMA_DOMAIN Class

class plasma_domain:
    ''' The plasma_domain class contains the information about the plasma to be studied.
    
    self.Nx     - This is the number of grid cells that divide the cavity
    self.k00    - The characteristic wavelength
    self.p0plas - The cell at which the plasma edge is located
    self.wpe2   - The square of the plasma frequency
    self.wc     - The cyclotron frequency
    self.ν      - The real-world frequency (in GHz) assigned to k00.
    self.νrel   - The quick unit change from rad/point (k00) to GHz (ν)
    self.ΔR_Δx  - Length (m) corresponding to each grid point (point^{-1})
    self.Δt_Δx  - Time (s) corresponding to each iteration step / grid point (point^{-1}) (magic step)
    
    '''
    
    def __init__(self,wpe2,wc,Nx=2100,p0plas=90,k00=0.025,ν=84,info=False):
        
        self.k00 = k00
        
        if len(wpe2)!=len(wc): raise Exception("The spatial domains of wpe2 and wc do not coincide!")
        self.Nx = Nx if Nx else len(wpe2)
        self.wpe2 = np.array(copy.deepcopy(wpe2))
        self.wc = np.array(copy.deepcopy(wc))
        
        self.p0plas = p0plas if p0plas else Nx-list((np.ceil(wpe2)[::-1])).index(0)-1
        
        # -- Dimensional equivalencies
        self.ν     = ν
        self.νrel  = ν/k00
        self.ΔR_Δx = (k00*299792458)/(2*np.pi*ν*1E9)
        self.Δt_Δx = k00/(2*np.pi*ν)
        

        
        if info: self.print_info()
        
    def print_info(self):
        print('Plasma Parameters:')
        print('Nx = %i , p0plas = %i'%(self.Nx,self.p0plas))
        print('k00 = %.3f equivalent to %.1f GHz'%(self.k00,self.ν))
        
    # -- Wave-related functions
    
    def Xmode_N(self,ki):
        return np.array([cm.sqrt(1-(self.wpe2[i]*(ki*ki-self.wpe2[i])/ki**2)/(ki**2-self.wpe2[i]-self.wc[i]**2)) for i in range(self.Nx)]).real
    
    def cut_offLoc(self,ki):
        try: return list(self.Xmode_N(ki)).index(0)
        except ValueError: return None
    
    def plot(self,ki,kimin=None,kimax=None,ax=None):
    
        mode='axes'
        if ax==None:
            fig, ax = plt.subplots(figsize=(6,3))
            mode='plot'

        ax.set_xlabel("x (points)")
        ax.axvline(x=self.p0plas,linewidth=0.5,color='lightgray')
        ax.scatter(range(self.Nx),self.Xmode_N(ki),marker='.',s=1,c="darkorange",label="$N_X$")
        if kimin: ax.scatter(range(self.Nx),self.Xmode_N(kimin),marker='.',s=1,c="darkorange",alpha=0.05)
        if kimax: ax.scatter(range(self.Nx),self.Xmode_N(kimax),marker='.',s=1,c="darkorange",alpha=0.05)
        
        ax.plot(self.wpe2/np.max(self.wpe2),c="slateblue",label="$\omega_{pe}^2/\omega^2\propto n_e$")
        ax.plot(self.wc**2/np.max(self.wc**2),c="darkturquoise",label="$\omega_{c}^2/\omega^2\propto B^2$")
        ax.set_ylim(-0.1,1.3*np.max([1.15,np.nanmean(self.Xmode_N(ki))]))
        ax.legend()
    
        if mode=='plot':
            plt.show()
        else: return ax
    
    def plot_report(self,ki=None,figsize=(5,3),savefile=None,ax=None):
        
        mode='axes'
        if ax==None:
            fig, ax = plt.subplots(figsize=figsize)
            mode='plot'
        
        R_ = [(x-self.p0plas)*self.ΔR_Δx for x in range(self.Nx)]

        ax.plot(R_,self.wc*self.νrel,label='$ω_{ce}$',c='darkturquoise') # remove *νrel for points^{-1}
        ax.plot(R_,np.sqrt(self.wpe2)*self.νrel,label='$ω_{pe}$',c='slateblue') # remove *νrel for points^{-1}
        ax.set_xlabel('Distance into plasma (m)')
        ax.set_ylabel('Frequency (GHz)')
        ax.tick_params(axis='x',labelsize=8)
        ax.tick_params(axis='y',labelsize=8)
        ax.invert_xaxis()

        if ki:
            axN=ax.twinx()
            axN.scatter(R_,self.Xmode_N(ki),label='$N_X$',c='darkorange',s=0.5)
            axN.set_ylim(0,2.5)
            axN.tick_params(axis='y',labelsize=8,labelcolor='darkorange')
            axN.set_ylabel('Refractive index $N$',c='darkorange')

        ax.legend(loc='lower left')

        if savefile: plt.savefig(savefile,bbox_inches='tight',transparent=True,dpi=500)

        if mode=='plot':
            plt.show()
        else: return ax
        
    # -- X-mode Cut-off frequencies
    
    def wco_up(self,x):
        return 0.5*(self.wc[x]+np.sqrt(self.wc[x]**2+4*self.wpe2[x]))
    
    def wco_low(self,x):
        return 0.5*(-self.wc[x]+np.sqrt(self.wc[x]**2+4*self.wpe2[x]))


Alternative initial parameter definition to 'zoom in'.

<h3>Sources</h3>

In [3]:
# -- Probing wavenumbers -- #

def OGkiSweep(t,ki0,sr=5000,srt=300000):  # Original Mathematica Source
    return ki0*(0.995+0.01*(t-sr)/srt)

def k0i(t,ki):           # Constant probing frequency
    return ki

# -- Source definitions -- #

def OGsource(t,ki):       # Original Mathematica Source
    return 0.5*(np.tanh(0.000125*(t-250))-np.tanh(0.000125*(t-125000)))*np.sin(t*ki)

def sinSource(t,ki):      # Pure sinusoidal source
    return np.sin(t*ki) 

def gaussSource(t,ki,σ=10): # Gaussian excitation
    μ=5*σ
    return np.exp(-(t-μ)**2/(2*σ**2))#/(σ*np.sqrt(2*np.pi))

<h3>Evolution of an X-mode wave in plasma</h3>

From Muriel COLIN's 2001 pHD thesis (page 42) we know that the equations governing the evolution of the electric field are the following.

\begin{align}
\left[\frac{\partial^2}{\partial t^2}+\omega_{pe}^2(x)\right]E_x&=-\omega_{pe}^2(x)B_0(x)v_y&(1)\\
\left[\frac{\partial^2}{\partial t^2}-c^2\frac{\partial^2}{\partial x^2}+\omega_{pe}^2(x)\right]E_y&=\omega_{pe}^2(x)B_0(x)v_x&(2)\\
\frac{\partial}{\partial t}v_x&=-\frac{e}{m_e}E_x-\omega_{ce}(x)v_y&(3)\\
\frac{\partial}{\partial t}v_y&=-\frac{e}{m_e}E_y+\omega_{ce}(x)v_x&(4)
\end{align}

where both components of $E$ and $v$ are dependent of $x$ and $t$.

To solve the four coupled equations numerically we will use Runge-Kutta 4 on the two equations on the velocity and Numerov's method for the second order differential equations on the components of the electric field. We are not aware of any high-order numerical scheme to resolve the equation on $E_y$, given that it also has the second derivative on the spatial coordinate. We will be using a custom second-order method, which does not behave as well numerically as the other two (fourth-order methods), hence it will be the one limiting the numerical performance. As the four equations are coupled and we will use a different scheme for each, the numerical scheme will be typed by hand.

Additionally we will work with the normalised electric field $\frac{\vec{E}}{B_0(x)}$, which we will still denote with ```E_```. Notice how we could have added a new dimension to the ```E_``` and ```v_``` variables to keep a record of the time evolution at all points for both components. However, as ```Nx=1E3```and ```Nt=1E5```, this would have implied that each variable occupies the order of 1 GB at single precision. We define instead ```E_hist```.

We define now the coefficients for the numerical schemes. 

Equation (1), under Numerov's method, and approximating the contribution of the velocity, becomes as follows.

$$\left(\frac{E_x(x_i)}{B_0(x_i)}\right)^{(t_n)}=\underbrace{\frac{2-\frac{5}{6}\omega_{pe}^2(x_i)}{1+\frac{1}{12}\omega_{pe}^2(x_i)}}_{\texttt{coef1Ex}}\left(\frac{E_x(x_i)}{B_0(x_i)}\right)^{(t_{n-1})}-\left(\frac{E_x(x_i)}{B_0(x_i)}\right)^{(t_{n-2})}-\underbrace{\frac{\omega_{pe}^2(x_i)}{1+\frac{1}{12}\omega_{pe}^2(x_i)}}_{\texttt{coef1Vy}}v_y(x_i)^{(t_{n-1})}\ .$$

Further, the second equation under the second order numerical scheme is,

$$\left(\frac{E_y(x_i)}{B_0(x_i)}\right)^{(t_n)}=\underbrace{\frac{1}{1+\frac{1}{2}\omega_{pe}^2(x_i)}}_{\texttt{coef2Ey}}\left[\left(\frac{E_y(x_{i-1})}{B_0(x_{i-1})}\right)^{(t_{n-1})}+\left(\frac{E_y(x_{i+1})}{B_0(x_{i+1})}\right)^{(t_{n-1})}\right]-\left(\frac{E_y(x_{i})}{B_0(x_{i})}\right)^{(t_{n-2})}+\underbrace{\frac{\omega_{pe}^2(x_i)}{1+\frac{1}{2}\omega_{pe}^2(x_i)}}_{\texttt{coef2Vx}}v_x(x_i)^{(t_{n-1})}\ .$$

Notice how, due to the second equation's numerical scheme, we are forced to keep in memory three time steps of the $y$ component of the normalised electric field. For simplicity, we will define ```Eo_``` (old) and ```Evo_``` (very old) following the notation of the original code. This is not necessary for the velocity.

In the original code ```coef1Ex```, ```coef1Vy```, ```coef2Ey```, and ```coef2Vx``` are called ```ctex```, ```ctvy```, ```cDtsDx```, and ```wpe2sDtwp``` respectively, where ```D```stands for $\Delta$ (In mathematica, as in python, one can use latex in variable names). Notice that all these coeficients are time-independent, so we can define them before resolution of the numerical scheme.

As mentioned, the two last equations follow the usual RK4, which we don't feel is necessary explaining here.

In [4]:
class wavePlasma:
    ''' We define the wave class to contain all the information necessary for the wave propagation simulation.
    
    self.plasma - The plasma_domain instance in which the wave will propagate.
    self.ki     - A one-argument function. Provides the incident wavenumber of the wave with respect to t.
    self.sExc   - A two-argument funtion. Provides the excitation that will generate the wave depending on t and wavenumber ki.
    self.sPos   - The grid position at which the source will be located
    self.sDir   - The direction of wave propagation
    self.E_     - A three dimensional array of shape (3,2,Nx). Contains 3 time-consecutive values of the two components of the electric field in the plasma domain.
    self.Hist   - A three dimensional array of shape (4,2,Nt), where Nt is the cumulated time iterations of wave propagation in the plasma. 
                  self.Hist[0] and self.Hist[-1] will be the electric field and velocity components of the plasma simulation at position self.hPos.
                  self.Hist[1] and self.Hist[2] the reflected and transmitted components (at positions sPos-2 and -1).
    self.hPos   - A parameter specifying the position at which the history is written (see self.Hist).
    
    '''    
    def __init__(self,plasma,ki,sExc,sPos = 'default',sDir = 'r',E_ = 'zeros',v_ = 'zeros',Hist = None,hPos = 110):
        
        self.plasma = plasma
        if str(type(ki))!="<class 'function'>": raise Exception("The wavenumber must be a function of t!")
        self.ki = ki
        if str(type(sExc))!="<class 'function'>": raise Exception("The source must be a function of t and ki!")
        self.sExc = sExc
        
        if sPos=='default': self.sPos = self.plasma.p0plas-10
        elif type(sPos)!=int: raise Exception("Source position must be an int!")
        elif sPos>=self.plasma.Nx or sPos<0: raise Exception("Source located outside cavity.")
        else: self.sPos = sPos
        
        self.sDir = sDir
        
        if E_ == 'zeros': E_ = np.zeros((3,2,self.plasma.Nx))
        elif np.shape(E_)!=(3,2,self.plasma.Nx): raise Exception('The shape of E_ should be (3,2,Nx)!')
        self.E_ = E_
        
        if v_ == 'zeros': v_ = np.zeros((2,self.plasma.Nx))
        elif np.shape(v_)!=(2,self.plasma.Nx): raise Exception('The shape of v_ should be (2,Nx)!')
        self.v_ = v_
        
        self.Hist = np.array([[[],[]],[[],[]],[[],[]],[[],[]]])
        if Hist: 
            if np.shape(Hist)[:-1]!=(4,2): raise Exception('Hist array has to have dimension (4,2,Nt)!')
            else: self.Hist = Hist
            
        self.Nt = np.shape(self.Hist)[-1]
        self.hPos = hPos
        
        
    def reset(self):
        self.E_ = np.zeros((3,2,self.plasma.Nx))
        self.v_ = np.zeros((2,self.plasma.Nx))
        self.Hist = np.array([[[],[]],[[],[]],[[],[]],[[],[]]])
        
    def plot_wave(self,ax=None):

        mode='axes'
        if ax==None:
            fig, ax = plt.subplots(figsize=(6,3))
            mode='plot'
    
        ax.scatter(self.sPos,0,marker='s',color='black')

        # -- Electric field plot --
        ax.set_ylim(-1.1,1.1)
        ax.plot(self.E_[0][0]/np.max(np.abs(self.E_[0][0])),label="$E_x$")
        ax.plot(self.E_[0][1]/np.max(np.abs(self.E_[0][1])),label="$E_y$")
        ax.set_yticks([])
        ax.legend(loc="lower right")
    
        if mode=='plot':
            plt.show()
        else: return ax
    
    def plot_wavePlasma(self,t,kimin=None,kimax=None):
        
        fig,axs=plt.subplots(2,1,figsize=(6,3.6),sharex=True)
        fig.suptitle("t = "+str(t))
        
        # -- Plasma cavity plot --
        axs[1]=self.plasma.plot(self.ki(t),ax=axs[1],kimin=kimin,kimax=kimax)
        
        # -- Electric field plot --
        axs[0]=self.plot_wave(axs[0])
        
        plt.show()
        
    def waveEvPlasma(self,Nt=10000,plot=True,Hist=False,recordKinetic=[]):
        
        if len(recordKinetic): 
            if np.shape(recordKinetic)[1]!=2: raise Exception('recordKinetic must be a list of two-tuples.')
            v_Kin   = []
            for i in range(len(recordKinetic)):
                v_Kin.append(np.zeros((recordKinetic[i][1],2,self.plasma.Nx)))
        
        # -- Source computation
        s=[self.sExc(t,self.ki(t)) for t in range(self.Nt,Nt+self.Nt)]
        kimin,kimax=freq_minmax(s)
        
        # -- Numerical schemes coefficients
        coef1Ex = (2-(5*self.plasma.wpe2)/6)/(1+self.plasma.wpe2/12)   # ctex
        coef1Vy = (self.plasma.wpe2)/(1+self.plasma.wpe2/12)           # ctvy
        coef2Ey = 1/(1+self.plasma.wpe2/2)                             # cΔtsΔx
        coef2Vx = self.plasma.wpe2*coef2Ey                             # wpe2sΔtwp
    
        startRunTime=timer()
    
        for t in range(Nt):          # -- Time loop -- 
        
            # -- Transparent boundary condition -- 2nd eq numerical scheme limited at the boundaries
            self.E_[0][0,0] = 0
            self.E_[0][1,0] = coef2Ey[0]*(self.E_[2][1,0]+self.E_[1][1,1])-self.E_[2][1,0]
    
            for x in range(1,self.plasma.Nx-1):     # -- Space loop --
        
                # -- Normalised electric field
                self.E_[0][0,x] = coef1Ex[x]*self.E_[1][0,x]-self.E_[2][0,x]-coef1Vy[x]*self.v_[1,x]
                self.E_[0][1,x] = coef2Ey[x]*(self.E_[1][1,x-1]+self.E_[1][1,x+1])-self.E_[2][1,x]+coef2Vx[x]*self.v_[0,x]
                
                # -- Perturbation velocity -- RK4
                k1 = -self.plasma.wc[x]*(self.E_[1][0,x]+self.v_[1,x])                             # k for x component
                l1 = -self.plasma.wc[x]*(self.E_[1][1,x]-self.v_[0,x])                             # l for y component
                k2 = -self.plasma.wc[x]*(0.5*(self.E_[1][0,x]+self.E_[0][0,x])+self.v_[1,x]+0.5*l1)
                l2 = -self.plasma.wc[x]*(0.5*(self.E_[1][1,x]+self.E_[0][1,x])-self.v_[0,x]-0.5*k1)
                k3 = -self.plasma.wc[x]*(0.5*(self.E_[1][0,x]+self.E_[0][0,x])+self.v_[1,x]+0.5*l2)
                l3 = -self.plasma.wc[x]*(0.5*(self.E_[1][1,x]+self.E_[0][1,x])-self.v_[0,x]-0.5*k2)
                k4 = -self.plasma.wc[x]*(self.E_[0][0,x]+self.v_[1,x]+l3)
                l4 = -self.plasma.wc[x]*(self.E_[0][1,x]-self.v_[0,x]-k3)
                self.v_[0,x] += (1/6)*(k1+2*k2+2*k3+k4)
                self.v_[1,x] += (1/6)*(l1+2*l2+2*l3+l4)
    
            # -- Transparent boundary condition -- 
            self.E_[0][0,-1] = 0
            self.E_[0][1,-1] = coef2Ey[-1]*(self.E_[1][1,-2]+self.E_[2][1,-1])-self.E_[2][1,-1]
    
            # -- We make history --
            if Hist:
                Histt = [[[self.E_[0][0,self.hPos]],[self.E_[0][1,self.hPos]]],
                         [[self.E_[0][0,self.sPos-2]],[self.E_[0][1,self.sPos-2]]],
                         [[self.E_[0][0,-1]],[self.E_[0][1,-1]]],
                         [[self.v_[0,self.hPos]],[self.v_[1,self.hPos]]]]
                self.Hist = np.append(self.Hist,Histt,axis=2)
            
                for i in range(len(recordKinetic)):
                    if t>=recordKinetic[i][0] and t<np.sum(recordKinetic[i]):
                        v_Kin[i][t-recordKinetic[i][0]] = copy.deepcopy(self.v_)
            
            #print(t)
            if (t+1)%20==0 and t>1:
                if plot:
                    clear_output(wait=True)
                    self.plot_wavePlasma(self.Nt+1,kimin,kimax)
                else:
                    runTime  = timer()-startRunTime
                    leftTime = runTime*(Nt-(t+1))/(t+1)
                    print('Time: %i / %i , %.2f, left %ih %im %is'%(t+1,Nt,(t+1)/Nt*100,leftTime//3600,(leftTime-3600*(leftTime//3600))//60,
                                                                    (leftTime-3600*(leftTime//3600)-60*(leftTime-3600*(leftTime//3600))//60)//1),end='\r')
        
            # -- Current E_ becomes Eo_ and so on --
            self.E_[2] = copy.deepcopy(self.E_[1])
            self.E_[1] = copy.deepcopy(self.E_[0])
        
            # -- Source addition -- According to UTS (What does it stand for?)
            self.E_[1][1][self.sPos]+=s[t]
            if self.sDir=='r': self.E_[2][1][self.sPos-1]+=s[t]
            elif self.sDir=='l': self.E_[2][1][self.sPos+1]+=s[t]
            
            self.Nt += 1
            
        if len(recordKinetic): return v_Kin
            
            
        ## -- Time of flight -- 
    
    def plot_TransReflect(self,interv=(None,None)):
        plot_TransReflect(self.Hist[2][1],self.Hist[1][1],interv)
        
    def time_of_flight(self,gauss=(True,True),Fast=True):
        SourceSignal = [self.sExc(t,self.ki(t)) for t in range(self.Nt)]
        RefSignal    = copy.deepcopy(self.Hist[1][1])
        return time_of_flight(SourceSignal,RefSignal,gauss=(True,True),Fast=True)

        
        

<h3>Signal description</h3>

To work with signals, both the excitation but also the transmitted and reflected waves. Most of these functions can be found defined elsewhere (mainly scipy.signal) but we prefered to include here the functions we will be needing.

In [5]:
# We develop a function to check whether the amplitude remains constant.
# We will verify with a pure sinusoidal wave.

def isAmpConstant(wave,tolerance=1e-4):
    
    ext=extrema(wave,tolerance)
    maxRelstd=np.std(ext[0][1])/np.average(ext[0][1])
    minRelstd=np.std(ext[1][1])/np.average(ext[1][1])
    
    return maxRelstd<tolerance and minRelstd<tolerance

# -- Signal Frequency Representation -- #

def plot_freqSpectrum(timeSpectrum,timediv=0,freqdiv=2,ki=None):
    N=len(timeSpectrum)
    if not timediv: timediv=N
    fig,axs=plt.subplots(1,2,figsize=(11,3))
    axs[0].set_title('Time spectrum')
    axs[0].plot(range(N)[:timediv],timeSpectrum[:timediv])
    axs[1].set_title('Frequency spectrum')
    if ki: axs[1].axvline(x=ki,linewidth=0.5,color='lightgray')
    axs[1].plot((2*np.pi)*fftfreq(N)[:N//freqdiv],np.abs(fft(timeSpectrum)[:N//freqdiv]))
    plt.show()

def freq_minmax(timeSpectrum):
    N=len(timeSpectrum)
    f_=np.abs(fft(timeSpectrum))[:N//2]
    fmax=np.max(f_)
    A=[1 if f>fmax*1e-4 else 0 for f in f_]
    return ((2*np.pi)*fftfreq(N)[list(A).index(1)],(2*np.pi)*fftfreq(N)[len(A)-list(A[::-1]).index(1)-1])


# -- Envelope tracing -- #

def roots(signal):
    root = []
    for t in range(1,len(signal)-1):
        if signal[t]*signal[t-1]<0: root.append(t-signal[t]/(signal[t]-signal[t-1]))
    return root

def extrema(signal,All=False,Fast=True,tol=1e-4):
    
    maxis = [[],[]]
    minis = [[],[]]
    
    if Fast:
        
        if All: extis = [[],[]] # For all extrema
    
        for i in range(1,len(signal)-1): # We ignore boundaries
        
            # i can represent both time or space, depending on the array dependence
            if (signal[i]>=signal[i-1] and signal[i]>signal[i+1] and np.abs(signal[i])>tol): 
                #if len(maxWave[0])>0 and maxWave[0][-1]==i-1: continue
                maxis[0].append(i)
                maxis[1].append(signal[i])
                if All:
                    extis[0].append(i)
                    extis[1].append(signal[i])
            
            if (signal[i]<=signal[i-1] and signal[i]<signal[i+1] and np.abs(signal[i])>tol): 
                #if len(minWave[0])>0 and minWave[0][-1]==i-1: continue
                minis[0].append(i)
                minis[1].append(signal[i])
                if All:
                    extis[0].append(i)
                    extis[1].append(signal[i])
        
    else:
        N  = len(signal)
        f  = make_smoothing_spline(range(N),signal)
        r_ = roots(f.derivative(1)(range(N)))
        fpp = f.derivative(2)
        for r in r_:
            if fpp(r)>0: 
                minis[0].append(r)
                minis[1].append(f(r))
            else:
                maxis[0].append(r)
                maxis[1].append(f(r))
        extis = [r_,f(r_)]
        
    return (maxis,minis,extis) if All else (maxis,minis)
            

def envelope(signal,which='both',lam=None,normax=None,Fast=True,reltol=1e-4):
    if which=='both': signal=np.abs(signal) # We use both maxima and minima to compute the envelope
    if normax=='y' or normax=='both': signal=signal/np.max(np.abs(signal))
    
    maxis,minis=extrema(signal,Fast=Fast,tol=np.max(np.abs(signal))*reltol)
    if which=='both' or which=='upper': selec=copy.deepcopy(maxis)
    elif which=='lower': selec=copy.deepcopy(minis)
    else: raise Exception('Extrema selection wrong. which can be \'both\', \'upper\', or \'lower\'.')
    if normax=='x' or normax=='both': selec[0]=np.array(selec[0])/len(signal)
    
    # -- We make sure that the envelope is correctly recovered
    N   = len(signal)
    var = np.linspace(0,1,N)
    r_ = []
    skip = 0
    while len(r_)==0:
        skip += 1
        env   = make_smoothing_spline(selec[0][::3*skip],selec[1][::3*skip],lam=lam)
        r_    = roots(env.derivative(1)(var))
        
    return env

def peakloc_gaussPulse(signal,Fast=True):
    # We begin by obtaining the x-normalised envelope
    N   = len(signal)
    var = np.linspace(0,1,N)
    
    # -- We obtain the second derivative
    D2signal = envelope(signal,which='upper',normax='x',Fast=Fast).derivative(2)(var)
    minidx=list(D2signal).index(np.min(D2signal))
    rootsignal=roots(D2signal)
    
    # -- Average of the two interesting inflection points
    inflxloc = []
    for i in range(len(rootsignal)):
        if rootsignal[i]>minidx:
            inflxloc.append(rootsignal[i-1])
            inflxloc.append(rootsignal[i])
            break
    
    return np.average(inflxloc)

def peakloc_Pulse(signal,Fast=True):
    # We begin by obtaining the x-normalised envelope
    N   = len(signal)
    var = np.linspace(0,1,N)

    # -- We make sure that the envelope is correctly recovered
    #r_ = []
    #skip = 0
    #while len(r_)==0:
    #    skip += 1
    #    env   = envelope(signal[::skip],which='upper',normax='x',Fast=True)
    #    r_    = roots(env.derivative(1)(var))
    
    # -- We switch to the detailed envelope if desired 
    env   = envelope(signal,which='upper',normax='x',Fast=Fast)
    r_    = roots(env.derivative(1)(var))
    
    # -- We obtain the absolute maximum
    here  = list(signal).index(np.max(signal))
    dist  = []
    for r in r_: dist.append(np.abs(r-here))
    
    return r_[list(dist).index(np.min(dist))] 

def wavenumber_atroots(signal):
    # We obtain the roots and the wavenumber values
    r_=roots(signal)
    k_=[0]
    
    for i in range(1,len(r_)-1):
        k_.append(2*np.pi/(r_[i+1]-r_[i-1]))
    
    avg = np.average(k_)
    k_.append(avg)
    k_[0] = avg
    
    return (r_,k_)

def wavenumberloc_Pulse(signal,wavenumber):
    r_,k_ = wavenumber_atroots(signal)
    
    # We find the positions of the sought wavenumber
    kpos1 = roots(np.array(k_)-wavenumber)
    kpos2 = [(r_[int(kp1+1)]-r_[int(kp1)])*(kp1-int(kp1))+r_[int(kp1)] for kp1 in kpos1]

    #Envsignal = envelope(signal,normax='x')(var)
    
    return kpos2
        

<h3>Time of flight</h3>

In [6]:
def plot_TransReflect(trans,ref,interv=(None,None)):
    fig,ax=plt.subplots(figsize=(6,4))
    ax.set_xlabel("time")
    
    if not interv[0]: xmin = 0
    else: xmin = interv[0]
    if not interv[1]: xmax = len(ref)
    else: xmax = interv[1]
    
    ax.plot(range(xmin,xmax),trans[interv[0]:interv[1]],label="Transmitted",c='darkturquoise')
    ax.plot(range(xmin,xmax),ref[interv[0]:interv[1]],label="Reflected",c='indianred',alpha=0.9)
    
    fig.legend()
    plt.show()
    
    
def time_of_flight(SourceSignal,RefSignal,gauss=(True,True),Fast=True):
    if len(SourceSignal)!=len(RefSignal): 
        raise Exception('Source and reflection time domains do not coincide!')
    else: N=len(SourceSignal)
    
    timeSource = peakloc_gaussPulse(SourceSignal,Fast=Fast) if gauss[0] else peakloc_Pulse(SourceSignal,Fast=Fast)
    timeReflected = peakloc_gaussPulse(RefSignal,Fast=Fast) if gauss[1] else peakloc_Pulse(RefSignal,Fast=Fast)
    
    return timeReflected-timeSource
    

<h3>Plasmas with linear profiles</h3>

We will define here a plasma_domain subclass to work more easily with linear profiles. Let us begin by introducing the theoretical background to our work developed by Roland Sabot. We consider a plasma with linear density profiles and linear magnetic dependency.

\begin{align}
    \omega_{pe}^2(x)&=\omega_{p0}^2\frac{x}{L_n}\\
    \omega_{c}^2(x)&=\omega_{ca}^2\left(1+\frac{2x}{L_B}\right)
\end{align}

where $L_n$ and $L_B$ are called respectively the density and magnetic field gradient lengths. Note that $\omega_{p0}$ and $\omega_{ca}$ are reference frequencies. 

Let us discuss the usual values of these quantities. In the tokamatk WEST, the egde magnetic field is around $B_a=3$ T, meaning that $\omega_{ca} \approx 524\ \text{G}\frac{\text{rad}}{\text{s}}$, or a frequency of $\nu_{ca}=84$ GHz. This means that we will choose the probing frequency to be between $81$ and $87$ GHz, but more on that later. Similarly, the electron density at the edge is around $n_e=5\cdot10^{19}\ \text{m}^{-3}$, meaning that the pulsation is $\omega_{p0}\approx400\ \text{G}\frac{\text{rad}}{\text{s}}$, or a frequency of $\nu_{p}=63$ GHz. Hence we will take

\begin{equation}
    \frac{\omega_{ca}^2}{\omega_{p0}^2}=\frac{16}{9}
\end{equation}

Now, to retain the same level of spatial detail of the wave we will not change the numerical value of ```k00```, but we can choose which frequency it represents. To keep it centered, we will take $k_{00}\equiv84$ GHz $=\nu_{ca}$ (frequency and wavenumber equivalence as explained above). Hence, in our code we will set:
    
    wca2 = k00**2
    wp02 = (9/16)*wca2

This leaves us needing only to set the values of $L_n$ and $L_B$. To do so we introduce the *gradient control parameter* $\eta_0$, which is a ratio describing the cavity that depends on the four values:

\begin{equation}
    \eta_0=2\left(\frac{\omega_{ca}^2}{L_B}\right)\left(\frac{\omega_{p0}^2}{L_n}\right)^{-1}=2\left(\frac{\omega_{ca}^2}{\omega_{p0}^2}\right)\left(\frac{L_n}{L_B}\right)\quad\in[0,\infty]
\end{equation}

It can be proved that the time of flight for a probing wave at cutoff frequency $\omega_{c0}$ for a cutoff at the very beginning of the cavity ($x_c\to0$) can be given by the following expression

\begin{equation}
    \tau\left(\omega_{c0},x_c\to0\right)=\frac{2L_B}{c}\cdot\eta_0\left(\frac{\sqrt{2-\eta_0}}{(1+\eta_0)^{3/2}}-\frac{\sinh^{-1}\sqrt{1+\eta_0}}{(1+\eta_0)^2}\right)
\end{equation}

This is the expression that we want to numerically verify for $\eta_0\in[0,10]$, or in other words,

\begin{equation}
    \eta_0=\frac{32}{9}\left(\frac{L_n}{L_B}\right)\quad\in[0,10]\quad\iff\quad \frac{L_n}{L_B}\in\left[0,\frac{90}{32}\right]
\end{equation}

Let us denote $L_n=\gamma L_B$ and $L_B=\beta N_x$, and let us study the values of $\gamma$ and $\beta$ that verify the two above conditions.

Now, as we can not numerically set $x_c=0$, or there would be no simulation, we will run it for $x_c=10$. With this, knowing that the cutoff frequency $\omega_{c0}$ is given by the expression below, we can write the following.
\begin{align}
    \omega_{c0}(x)&=\frac{1}{2}\omega_{c}(x)+\frac{1}{2}\sqrt{\omega_c^2(x)+4\omega_{pe}^2(x)}\\
    &=\frac{1}{2}\omega_{ca}\sqrt{1+\frac{2x}{L_B}}+\frac{1}{2}\sqrt{\omega_{ca}^2\left(1+\frac{2x}{L_B}\right)+4\omega_{p0}^2\frac{x}{L_n}}\\
    &=\omega_{p0}\left[\frac{2}{3}\sqrt{1+\frac{2x}{L_B}}+\sqrt{\frac{4}{9}\left(1+\frac{2x}{L_B}\right)+\frac{x}{L_n}}\right]
\end{align}

we want this value to be in the $[81,87]$ GHz interval as we mentioned before, so we limit the possible values of $L_n$ and $L_B$. The determination of the range of values that verify these two conditions simultaneously is rather difficult to compute analytically, so we have done so numerically.

**To conclude** we have introduced two new parameters, $\gamma$ and $\beta$. After setting the values of $\omega_{ca}$ and $\omega_{p0}$, $\gamma$ is directly proportional to the gradient control parameter $\eta_0$, which is the parameter against which we want to measure the goodness of the WKB approximation. $\beta$ is the scaling parameter of our plasma domain with respect to the position at which the magnetic field doubles. Also, the bigger $\beta$ is, the lower the probing frequency $\omega_{c0}(x)$ can be (more realistic), but the longer the simulation takes (due to the dependency of the time of flight with $\beta$). Among the numerically confirmed possible values of $\beta$, we choose $\beta=27.9$.

In [7]:
def wpe2_linear(x,wp02,Ln,p0plas):
    return wp02*(x-p0plas)/Ln if x>p0plas else 0

def wc_linear(x,wca2,LB,p0plas):
    return np.sqrt(wca2*(1+2*(x-p0plas)/LB))

def η0(wp02,wca2,Ln,LB):
    return 2*(wca2/wp02)*(Ln/LB)


class linearPlasma_domain(plasma_domain):
    
    def __init__(self,γ,β=27.9,wp02='default',wca2='default',Nx=2100,p0plas=90,k00=0.025,ν=84,info=False):
        
        if wp02=='default': wp02=(9/16)*k00**2
        if wca2=='default': wca2=k00**2
        
        super().__init__([wpe2_linear(x,wp02,γ*β*Nx,p0plas) for x in range(Nx)],
                         [wc_linear(x,wca2,β*Nx,p0plas) for x in range(Nx)],
                         Nx=Nx,p0plas=p0plas,k00=k00,ν=ν,info=info)
        
        self.η0 = η0(wp02,wca2,γ*β*Nx,β*Nx)

<h4>Results visualisation</h4>

In [8]:
# -- Convenience --
def str_dclp(value,ndecimals=1):
    return '%ip%i'%(value,np.sign(value)*np.ceil(np.round(value-int(value),ndecimals)*10**ndecimals))

def vmodTime(dataframeloc):
    return np.sqrt(dataframeloc.at['KineticTime'][0]**2+dataframeloc.at['KineticTime'][1]**2)

# -- Result plotting methods --

def wavenumber_info(dataframeloc,β=27.9,p0plas=90,zoomscale=1e-3,figsize=(4.8,2.1),dpi=200,savefile=None,GHz=False,seconds=False):
    
    ref = dataframeloc.at['Reflected']
    plas = linearPlasma_domain(dataframeloc.at['gamma'],β)
    try: wco = plas.wco_up(dataframeloc.at['xc'])
    except: wco = dataframeloc.at['probFrec (points)']
    r_,k_ = wavenumber_atroots(ref)
    kloc_ = wavenumberloc_Pulse(ref,wco)
    
    fact = plas.νrel if GHz else 1
    wco = wco*fact
    k_  = np.array(k_)*fact
    
    xfact = plas.Δt_Δx if seconds else 1
    r_ = np.array(r_)*xfact
    
    fig,ax = plt.subplots(figsize=figsize,dpi=dpi)

    for kloc in kloc_:
        ax.axvline(kloc*xfact,linewidth=1,c='lightgray')

    ax.axhline(wco,linestyle='dotted',linewidth=1,c='gray')
    ax.plot(r_,k_,c='navy',linewidth=0.5)
    if GHz: ax.set_ylabel('Frequency (GHz)',fontsize=10,c='navy')
    else: ax.set_ylabel('$k\ \equiv\ \omega$ (rad/point)',fontsize=10,c='navy')
    if seconds: ax.set_xlabel('Time (ns)')
    else: ax.set_xlabel('Time iteration')
    ax.set_ylim(wco*(1-2.1*zoomscale/fact),wco*(1+2.1*zoomscale/fact))
    ax.set_yticks(wco+(np.arange(5)-2)*int(zoomscale/10**np.floor(np.log10(zoomscale)))*10**np.floor(np.log10(zoomscale)),
                  (np.arange(5)-2)*int(zoomscale/10**np.floor(np.log10(zoomscale))))
    ax.tick_params(axis='y', colors='navy',labelsize=8)
    ax.tick_params(axis='x',labelsize=8)
    ax.text(0, 1.02, "%.0e+%g"%(10**np.floor(np.log10(zoomscale)),wco),transform=ax.transAxes,fontsize=7)
    
    ax2 = ax.twinx()
    ax2.set_ylim(-1.05,1.05)

    ax2.plot(np.arange(len(ref))*xfact,ref,c='indianred',alpha=0.3)
    peakloc = peakloc_Pulse(ref)*xfact
    ax2.axvline(peakloc_gaussPulse(ref)*xfact,c='darkorange',label='Gauss peak',linestyle='-.',linewidth=1)
    ax2.axvline(peakloc,c='darkgreen',label='Envelope max',linestyle='--',linewidth=1)
    ax2.set_ylabel('Reflected signal rel. amp.',c='indianred')
    ax2.tick_params(axis='y', colors='indianred',labelsize=8)
    ax2.legend(fontsize=9)
    
    # k at peak
    peakloc2 = roots(r_-peakloc)
    katpeak = (k_[int(peakloc2[0]+1)]-k_[int(peakloc2[0])])*(peakloc2[0]-int(peakloc2[0]))+k_[int(peakloc2[0])]
    ax.scatter(peakloc,katpeak,marker='x',s=9,c='darkgreen')
    t=plt.text(0.03,0.1,'Relative $k$ error at peak: %.2e'%((katpeak-wco)/wco),backgroundcolor='white',transform=ax.transAxes,fontsize=9)
    t.set_bbox(dict(facecolor='white', alpha=0.8,edgecolor='lightgray',boxstyle='round'))
    
    try: fig.suptitle('$x_c=$ %i, $\eta_0=$ %.2f'%(dataframeloc.at['xc']-p0plas,dataframeloc.at['eta0']) )
    except: pass
    
    if savefile: plt.savefig(savefile,bbox_inches='tight',transparent=True,dpi=500)

    plt.show()
    
def kinetic_info(dataframeloc,β=27.9,figsize=(5.1,4.7),dpi=200,savefile=None,seconds=False):
    
    #fig,axs = plt.subplots(1,2,figsize=(11,3),dpi=dpi)
    fig,axs = plt.subplots(2,1,figsize=figsize,dpi=dpi,height_ratios=[1.2,2])
    
    plas = linearPlasma_domain(dataframeloc.at['gamma'],β)
    xfact = plas.Δt_Δx if seconds else 1

    velcolors = ['indigo','darkmagenta','mediumorchid','orchid','thistle']

    axs[0].plot(np.arange(len(vmodTime(dataframeloc)))*xfact,vmodTime(dataframeloc),c=velcolors[0],zorder=10)
    axs[0].set_ylabel('$|\\vec{v}|$ at probe (a.u.)',c=velcolors[0])
    axs[0].set_yticks([])
    if seconds: axs[0].set_xlabel('Time (ns)')
    else: axs[0].set_xlabel('Time iteration')
    axs[0].tick_params(axis='y', colors=velcolors[0])
    ax0=axs[0].twinx()
    ax0.plot(np.arange(len(dataframeloc.at['Source']))*xfact,dataframeloc.at['Source'],c='cornflowerblue',alpha=0.3)
    ax0.set_ylabel('Source',c='cornflowerblue')
    ax0.tick_params(axis='y', colors='cornflowerblue')

    for i in range(len(dataframeloc.at['recordKinetic'])):
        axs[0].axvline(dataframeloc.at['recordKinetic'][i][0]*xfact,linewidth=1,c=velcolors[i+1])
        axs[0].axvline((dataframeloc.at['recordKinetic'][i][0]+dataframeloc.at['recordKinetic'][i][1])*xfact,linewidth=1,c=velcolors[i+1])
    
    axs[1].set_ylabel('$|\\vec{v}|$ averaged over $T_c$ (a.u.)')

    axs[1].axvline(x=plas.p0plas,linewidth=0.5,color='lightgray',label='Plasma edge')
    axs[1].axvline(dataframeloc.at['xc'],color='red',alpha=0.5,linestyle='dotted',label='Cutoff')
    axs[1].axvline(110,c=velcolors[0],linestyle='dashed',linewidth=0.5,label='Kinetic time probe')

    for i in range(len(dataframeloc.at['KineticSpace'])):
        vmod=[]
        for t in range(len(dataframeloc.at['KineticSpace'][i])):
            vmod.append(np.sqrt(dataframeloc.at['KineticSpace'][i][t][0]**2+dataframeloc.at['KineticSpace'][i][t][1]**2))
        axs[1].plot(np.average(vmod,axis=0),c=velcolors[i+1],label='$|\\vec{v}|_{\mathrm{avg}}$ at t=%i'%(dataframeloc.at['recordKinetic'][i][0]))
    
    #axs[1].legend(title='$x_c=0.4\lambda$, $\eta_0$ = %.2f'%(dataframeloc.at['eta0']))
    axs[1].legend(title='$x_c = %.1f\lambda$, $\eta_0$ = %.2f'%((dataframeloc.at['xc']-plas.p0plas)*plas.k00/(2*np.pi),dataframeloc.at['eta0']))
    axs[1].set_xlabel('Cavity position')
    axs[1].set_yticks([])

    print('velocity units!')
    fig.tight_layout()
    
    if savefile: plt.savefig(savefile,bbox_inches='tight',transparent=True,dpi=500)
    plt.show()